In [ ]:
!git clone https://github.com/d4nst/RotNet.git

In [ ]:
from google.colab import files

print('Upload RotNet barcode models you want to convert to TFLite:')
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded model file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
%cd RotNet

In [ ]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
import numpy as np
from utils import angle_error


model = keras.models.load_model('/content/rotnet_barcode_view_resnet50_v2.hdf5',  # replace with your model name
                                custom_objects={'angle_error': angle_error})

# Convert Keras model to ConcreteFunction
full_model = tf.function(lambda x: model(x))
full_model = full_model.get_concrete_function(
    x=tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))

# Get frozen ConcreteFunction
frozen_func = convert_variables_to_constants_v2(full_model)
frozen_func.graph.as_graph_def()

# inspect the layers operations inside your frozen graph definition and see the name of its input and output tensors
layers = [op.name for op in frozen_func.graph.get_operations()]
print("-" * 50)
print("Frozen model layers: ")
for layer in layers:
    print(layer)

print("-" * 50)
print("Frozen model inputs: ")
print(frozen_func.inputs)
print("Frozen model outputs: ")
print(frozen_func.outputs)

# Save frozen graph from frozen ConcreteFunction to hard drive
# serialize the frozen graph and its text representation to disk.
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir="./frozen_models",
                  name="rotnet_barcode_frozen_graph.pb",
                  as_text=False)

#Optional
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir="./frozen_models",
                  name="rotnet_barcode_frozen_graph.pbtxt",
                  as_text=True)

In [ ]:
import tensorflow as tf
from utils import angle_error

model = tf.keras.models.load_model('/content/rotnet_barcode_view_resnet50_v2.hdf5', custom_objects={'angle_error':angle_error})
tf.saved_model.save(model, "models")
print("successfully saved keras model file to tensorflow model file")



---



Inspect models directory

In [ ]:
!ls -l models

Inspect frozen_models directory

In [ ]:
!ls -l frozen_models

zip folder for download purpose

In [ ]:
!zip -r /content/frozen_models.zip /content/RotNet/frozen_models
!zip -r /content/saved_models.zip /content/RotNet/models

Download the zipped model

In [ ]:
from google.colab import files

files.download("/content/frozen_models.zip")
files.download("/content/saved_models.zip")